In [21]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/vnb-foml-2024-hackathon/foml24_hackathon/sample_submission.csv
/kaggle/input/vnb-foml-2024-hackathon/foml24_hackathon/descriptions.txt
/kaggle/input/vnb-foml-2024-hackathon/foml24_hackathon/train.csv
/kaggle/input/vnb-foml-2024-hackathon/foml24_hackathon/test.csv


# **Data Pre-processing**

In [22]:
Train_data = pd.read_csv("/kaggle/input/vnb-foml-2024-hackathon/foml24_hackathon/train.csv", sep=",", skipinitialspace=True, engine='python')
Test_data = pd.read_csv("/kaggle/input/vnb-foml-2024-hackathon/foml24_hackathon/test.csv", sep=",", skipinitialspace=True, engine='python')

In [23]:
# Finding the columns which have 70% NaN values
null_columns = []
num_of_rows = Train_data.shape[0]
rows =  0.70 * num_of_rows
for column_name, column_data in Train_data.items():
    null_count = column_data.isnull().sum()
    if null_count >= rows:
        null_columns.append(column_name)
print(len(null_columns))
print(null_columns)

25
['CropFieldConfiguration', 'CultivatedAndWildArea', 'FarmClassification', 'FarmShedAreaSqft', 'FieldConstructionType', 'FieldShadeCover', 'FieldZoneLevel', 'HarvestStorageSqft', 'HasGreenHouse', 'HasPestControl', 'NaturalLakePresence', 'NumberGreenHouses', 'NumberOfFarmingZones', 'PartialIrrigationSystemCount', 'PerimeterGuardPlantsArea', 'PrimaryCropAreaSqft', 'PrimaryCropAreaSqft2', 'ReservoirType', 'ReservoirWithFilter', 'TaxOverdueStatus', 'TaxOverdueYear', 'TotalAreaSqft', 'TotalReservoirSize', 'UndergroundStorageSqft', 'WaterReservoirCount']


In [24]:
# Finding attributes with all distinct entries

different_value_column = []
num_of_rows = Train_data.shape[0]

#Checking for columns with different values
for column_name, column_data in Train_data.items():
  num_unique = Train_data[column_name].nunique()
  if(num_unique == num_of_rows):
    different_value_column.append(column_name)

#Printing the name and number of columns
print(len(different_value_column))
print(different_value_column)

1
['UID']


In [25]:
# Mapping the Target value {high, medium, low} to {1,0, -1}

for index,value in Train_data['Target'].items():
    if value == 'high':
        Train_data.at[index,'Target'] = 1
    if value == 'medium':
        Train_data.at[index,'Target'] = 0
    if value == 'low':
        Train_data.at[index,'Target'] = -1

display(Train_data)

,UID,AgriculturalPostalZone,AgricultureZoningCode,CropFieldConfiguration,CropSpeciesVariety,CultivatedAndWildArea,CultivatedAreaSqft1,DistrictId,FarmClassification,FarmEquipmentArea,...,TotalTaxAssessed,TotalValue,TownId,TypeOfIrrigationSystem,UndergroundStorageSqft,ValuationYear,WaterAccessPoints,WaterAccessPointsCalc,WaterReservoirCount,Target
0,12998,291674,0.0,NaN,3.0,NaN,1136.0,1.0,NaN,NaN,...,8636.716,456255.6,118.0,NaN,NaN,2018.0,2.0,2.0,NaN,1
1,20860,164397,28.0,NaN,4.0,NaN,2083.0,1.0,NaN,NaN,...,18464.292,996887.6,24.0,1.0,NaN,2018.0,3.0,3.0,1.0,0
2,75725,616532,0.0,NaN,2.0,NaN,922.0,1.0,NaN,NaN,...,15594.568,1043780.0,9.0,1.0,NaN,2018.0,1.0,1.0,NaN,0
3,106521,942111,43.0,NaN,7.0,NaN,NaN,1.0,NaN,NaN,...,8494.618,435734.8,114.0,NaN,NaN,2020.0,3.0,3.0,NaN,-1
4,99467,475557,38.0,NaN,3.0,NaN,2225.0,3.0,NaN,0.0,...,13517.284,885400.0,6.0,NaN,NaN,2020.0,4.0,4.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112564,26998,380364,41.0,NaN,2.0,NaN,1672.0,3.0,NaN,460.0,...,13467.652,1106200.0,47.0,NaN,NaN,2018.0,2.0,2.0,NaN,0
112565,135304,369784,24.0,NaN,4.0,NaN,3340.0,2.0,NaN,620.0,...,21732.964,1643115.8,7.0,NaN,NaN,2020.0,3.0,3.0,NaN,0
112566,153756,414700,37.0,NaN,4.0,NaN,NaN,1.0,NaN,NaN,...,25067.856,1464648.6,1.0,NaN,NaN,2020.0,4.0,4.0,NaN,1
112567,129907,566488,19.0,NaN,2.0,NaN,1178.0,1.0,NaN,NaN,...,10530.652,679200.0,1.0,1.0,NaN,2020.0,2.0,2.0,1.0,0


In [26]:
# replacing the NaN values with mean, or median in the training dataset

missing_value = []
for column_name, column_data in Train_data.items():
    if Train_data[column_name].isnull().any():
        missing_value.append(column_name)

def get_median(column_name):
    return Train_data[column_name].median()

def get_mean(column_name):
    return Train_data[column_name].mean()


missing_value_columns = [column_name for column_name in Train_data.columns if Train_data[column_name].isnull().any()]


for column_name in missing_value_columns:

    if Train_data[column_name].isnull().any():

        if Train_data[column_name].skew():
            replacing_value = get_median(column_name)
        else:
            replacing_value = get_mean(column_name)

#         Train_data[column_name].fillna(replacing_value, inplace=True)
        Train_data.fillna({column_name: replacing_value}, inplace=True)

print(Train_data.shape)

(112569, 58)


In [27]:
# replacing the NaN values with mean, or median in test dataset
missing_value_test = []
for column_name, column_data in Test_data.items():
    if Test_data[column_name].isnull().any():
        missing_value.append(column_name)

def get_median_test(column_name):
    return Test_data[column_name].median()

def get_mean_test(column_name):
    return Test_data[column_name].mean()


missing_value_columns = [column_name for column_name in Test_data.columns if Test_data[column_name].isnull().any()]


for column_name in missing_value_columns:

    if Test_data[column_name].isnull().any():

        if Test_data[column_name].skew():
            replacing_value = get_median_test(column_name)
        else:
            replacing_value = get_mean_test(column_name)

#         Train_data[column_name].fillna(replacing_value, inplace=True)
        Test_data.fillna({column_name: replacing_value}, inplace=True)

print(Test_data.shape)

(15921, 57)


In [28]:
# capturing the Test UID's for final submission file
data_uid = Test_data.filter(['UID'])
print (data_uid)

          UID
0      130000
1      129101
2      147876
3      122624
4      159920
...       ...
15916   99588
15917   86801
15918   68439
15919   13210
15920  151497

[15921 rows x 1 columns]


In [29]:
# Excluding the columns which have a 70% of the entries as NaN
columns_to_include = []
null_columns.append('UID')
for attr in Train_data.columns:
    if attr not in null_columns:
        columns_to_include.append(attr)

print(len(columns_to_include))

32


In [30]:
# Splitting the data features and labels

X_train = Train_data.filter(columns_to_include)
X_train = X_train.drop(columns = 'Target')

y_train = Train_data.filter(['Target'])

X_test = Test_data.filter(columns_to_include)

In [31]:
# converting to an numpy array

X_train = X_train.to_numpy()
y_train = y_train.to_numpy().flatten()
X_test = X_test.to_numpy()



In [32]:
# X = X.to_numpy()
# y = y.to_numpy().flatten()
# # X_test = X_test.to_numpy()
# from sklearn.model_selection import train_test_split

# # Assuming X and y are defined for train-test splitting
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
#Standardizing the features

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [34]:
y_train = y_train.astype(int)
# y_test = y_test.astype(int)

# **Implementing Gradient Boosting**

In [35]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pandas as pd
from imblearn.over_sampling import RandomOverSampler


ros = RandomOverSampler(random_state=42)

# Apply random oversampling to the training data
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)



gb_model = GradientBoostingClassifier(n_estimators=150,learning_rate=0.1,max_depth=5)
gb_model.fit(X_resampled, y_resampled)

# Making predictions 
y_train_pred = gb_model.predict(X_resampled)
y_test_pred = gb_model.predict(X_test)

# Calculating train accuracy
train_accuracy = accuracy_score(y_resampled, y_train_pred)



In [36]:
# test_accuracy = accuracy_score(y_test, y_test_pred)

print(train_accuracy)

0.5251526233448325


In [37]:
y_test_pred

array([ 0, -1, -1, ...,  0,  0,  0])

# **Saving the output in submission.csv**

In [38]:
#Saving the predicted values in a csv file

target_predicted_values = pd.DataFrame(y_test_pred, columns = ['Target'])
output_data = pd.concat([data_uid,target_predicted_values ], axis=1)

print(output_data)

          UID  Target
0      130000       0
1      129101      -1
2      147876      -1
3      122624       0
4      159920       0
...       ...     ...
15916   99588       1
15917   86801       1
15918   68439       0
15919   13210       0
15920  151497       0

[15921 rows x 2 columns]


In [39]:
output_data['Target'] = output_data['Target'].astype(str)

for index,value in output_data['Target'].items():
    if value == '1':
        output_data.at[index,'Target'] = 'high'
    if value == '0':
        output_data.at[index,'Target'] = 'medium'
    if value == '-1':
        output_data.at[index,'Target'] = 'low'

display(output_data)

,UID,Target
0,130000,medium
1,129101,low
2,147876,low
3,122624,medium
4,159920,medium
...,...,...
15916,99588,high
15917,86801,high
15918,68439,medium
15919,13210,medium


In [40]:
value_counts = output_data['Target'].value_counts()
value_counts

Target
medium    7479
low       4452
high      3990
Name: count, dtype: int64

In [41]:
output_data.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
# Last cell for running test cases

import argparse
def make_predictions(test_fname, predictions_fname):
    
    Test_data = pd.read_csv(test_fname)
    
    # fill NaN values in test data
    missing_value_test = []
    for column_name, column_data in Test_data.items():
        if Test_data[column_name].isnull().any():
            missing_value.append(column_name)

    missing_value_columns = [column_name for column_name in Test_data.columns if Test_data[column_name].isnull().any()]

    for column_name in missing_value_columns:
        if Test_data[column_name].isnull().any():
            if Test_data[column_name].skew():
                replacing_value = get_median_test(column_name)
            else:
                replacing_value = get_mean_test(column_name)
            Test_data.fillna({column_name: replacing_value}, inplace=True)
    
    test_uid = Test_data.filter(['UID'])
    
    X_test = Test_data.filter(columns_to_include)
    
    X_test = X_test.to_numpy()
    
    X_test = scaler.transform(X_test)
    
    y_test_pred = gb_model.predict(X_test)
        
    target_predicted_values_test = pd.DataFrame(y_test_pred, columns = ['Target'])
    output_data_test = pd.concat([test_uid,target_predicted_values_test ], axis=1)
    
    output_data_test['Target'] = output_data_test['Target'].astype(str)

    for index,value in output_data_test['Target'].items():
        if value == '1':
            output_data_test.at[index,'Target'] = 'high'
        if value == '0':
            output_data_test.at[index,'Target'] = 'medium'
        if value == '-1':
            output_data_test.at[index,'Target'] = 'low'
    
    output_data_test.to_csv(predictions_fname, index=False)
    


if __name__=="__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--train-file", type=str, help=’file path of train.csv’)
    parser.add_argument("--test-file", type=str, help=’file path of test.csv’)
    parser.add_argument("--predictions-file", type=str, help=’save path of predictions’)
    args = parser.parse_args()
    make_predictions(args.test_file, args.predictions_file)